In [1]:
#!/usr/bin/env python3.6
# -*- coding: utf-8 -*-

In [2]:
# Imports
import sys
import math
import numpy as np
from matplotlib import animation
from IPython.display import HTML
from matplotlib import pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'
import mpl_toolkits.mplot3d.axes3d as p3

In [3]:
np.random.seed(20)
np.set_printoptions(threshold=sys.maxsize)

In [4]:
%matplotlib inline

# Data

In [5]:
# Read data
path = '../../../dataset4_wbc/live_extraction/spot/step_0.08/accelerations_1m.csv'
motion = np.genfromtxt(path, delimiter=',', dtype=np.float64)
contacts = np.genfromtxt(path, delimiter=',', usecols=(29, 30, 31, 32), dtype=bool)
print("Motion data has a length of ", len(motion))

Motion data has a length of  14665


In [6]:
motion.shape

(14665, 33)

## Height-Force plots

In [7]:
def get_specific_cmd(dataset, fwd, side, rot):
    if abs(fwd): 
        return np.where(dataset[:, 1] > 0)[0] if fwd > 0 else np.where(dataset[:, 1] < 0)[0]
    if abs(side): 
        return np.where(dataset[:, 2] > 0)[0] if side > 0 else np.where(dataset[:, 2] < 0)[0]
    if abs(rot): 
        return np.where(dataset[:, 3] > 0)[0] if rot > 0 else np.where(dataset[:, 3] < 0)[0]

In [8]:
def get_swinging_motions(dataset, height=1):
    # rf min height (i.e swining motion)
    fl_min_height = np.where(dataset[:, 6] < height)[0]
    fr_min_height = np.where(dataset[:, 9] < height)[0]
    rl_min_height = np.where(dataset[:, 12] < height)[0]
    rr_min_height = np.where(dataset[:, 15] < height)[0]
            
    return fl_min_height,fr_min_height,rl_min_height,rr_min_height

# Dataset Preparation

In [9]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [10]:
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[3]
    q1 = Q[0]
    q2 = Q[1]
    q3 = Q[2]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [11]:
def create_com_dataset_absolute(dataset, footsteps, motion=None, debug=False):
    inputs = []
    labels = []
    velocity_x = 0
    velocity_y = 0
    velocity_yaw = 0
    
    for idx, footstep in enumerate(footsteps):
        if idx > 0:
            # Compute time difference between
            # current and next footsteps
            time_difference = abs(dataset[footsteps[idx], 0] - dataset[footsteps[idx-1], 0])
            
            if time_difference < 0.5:
                fl_moving = contacts[idx, 0]
                fr_moving = contacts[idx, 1]
                rl_moving = contacts[idx, 2]
                rr_moving = contacts[idx, 3]
                
                assert fr_moving == rl_moving
                assert rr_moving == fl_moving
                assert fr_moving != fl_moving

                # Compute rotation matrices from odometry orientation
                R_prev = quaternion_rotation_matrix(dataset[footsteps[idx-1], 19:23])
                R_curr = quaternion_rotation_matrix(dataset[footsteps[idx], 19:23])

                # Odom base vectors
                prev_base = np.array([[dataset[footsteps[idx-1], 16], dataset[footsteps[idx-1], 17], 0]])
                curr_base = np.array([[dataset[footsteps[idx], 16], dataset[footsteps[idx], 17], 0]])

                # Rotate odometry estimation
                rotated_prev_base = R_prev.T @ prev_base.T
                rotated_curr_base = R_prev.T @ curr_base.T
                base_displacement = rotated_curr_base - rotated_prev_base

                inputs.append(dataset[footsteps[idx-1], 1:4].tolist() +
                              dataset[footsteps[idx-1], 23:29].tolist() +
                              dataset[footsteps[idx-1], 4:16].tolist() + 
                              [fl_moving, fr_moving, rl_moving, rr_moving] +
                              dataset[footsteps[idx], 1:4].tolist())
                
                # Labels
                labels.append([base_displacement[0,0], base_displacement[1,0]])
                
    # Convert to numpy array
    inputs_numpy = np.array(inputs, dtype=object)
    labels_numpy = np.array(labels, dtype=object)
    
    return inputs_numpy, labels_numpy

# Stack datasets
X_motion_CoM_acceleration, Y_motion_CoM_acceleration = create_com_dataset_absolute(motion, [x for x in range(len(motion))])
print(X_motion_CoM_acceleration.shape)
print(Y_motion_CoM_acceleration.shape)

(14658, 28)
(14658, 2)


In [12]:
%store X_motion_CoM_acceleration
%store Y_motion_CoM_acceleration

Stored 'X_motion_CoM_acceleration' (ndarray)
Stored 'Y_motion_CoM_acceleration' (ndarray)


In [15]:
Y_motion_CoM_acceleration

array([[0.026565099063248, -0.0077308875998767235],
       [0.018461610297009656, 0.005742571233412264],
       [0.02137906496059827, -0.00555438165181954],
       [0.014715893516921597, 0.005942671876823624],
       [0.042027444427333194, -0.0030630497601293882],
       [0.021412987184760368, 0.005292879119309646],
       [0.04241131803752296, -0.003470918031865722],
       [0.02195550965404891, 0.004840491387568591],
       [0.03899108346459457, -0.0034050559360707666],
       [0.011767982462714599, 0.006524848989670318],
       [0.027020479526015606, -0.003705848555313196],
       [0.010564713403368842, 0.005784486579216264],
       [0.026442954097376514, -0.004107062137421735],
       [0.012469651573184137, 0.005618690254124203],
       [0.04038799623955924, -0.00218741507896425],
       [0.02347743094924226, 0.005312985710872908],
       [0.043170926204396665, -0.0025597736230906776],
       [0.022036497172564395, 0.004696007970295568],
       [0.03700396339996814, -0.002681380001